[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/037_charRNN/charRNN.ipynb)

# Generación de texto

En este post vamos a entrenar una `red neuronal recurrente` para generar texto, carácter a carácter, inspirado en [CharRNN](https://github.com/karpathy/char-rnn). Nuestra red neuronal recibirá como entrada una secuencia de letras y deberá dar como salida la siguiente letra (la cual añadiremos a las entradas para volver a generar un nuevo carácter). 

## Los datos

Lo primero que necesitamos para lograr nuestro objetivo es un conjunto de datos. En este caso, al querer generar texto, nos servirá con un archivo con mucho texto que queramos imitar. Para ello descargaremos *Don Quijote de la Mancha*, la obra principal del escritor Miguel de Cervantes y una de las más relevantes en la literatura castellana. 

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=34dc6a059f68f086d014a10ad24301662e0a9995daa627860770d15ccde41234
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import wget

wget.download('https://mymldatasets.s3.eu-de.cloud-object-storage.appdomain.cloud/el_quijote.txt')

'el_quijote.txt'

In [3]:
f = open("el_quijote.txt", "r", encoding='utf-8')
text = f.read()
text[:300], len(text)

('DON QUIJOTE DE LA MANCHA\nMiguel de Cervantes Saavedra\n\nPRIMERA PARTE\nCAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha\nEn un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, ada',
 1038397)

Tenemos alrededor de 1 millón de carácteres en nuestro dataset, suficientes para generar texto de manera convincente como si fuésemos el manco de Lepanto.

## Tokenización

Para poder darle este texto a nuestra red neuronal necesitamos transformarlo en números con los que podemos llevar a cabo las operaciones que tienen lugar en la red. Este proceso se conoce como `tokenización`. Existen muchas formas de llevar a cabo este proceso, en este caso simplemente sustituiremos cada carácter en nuestro texto por su posición en el siguiente vector de carácteres.

In [4]:
import string

print(string.printable)
all_characters = string.printable + "ñÑáÁéÉíÍóÓúÚ¿¡"
all_characters

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0cñÑáÁéÉíÍóÓúÚ¿¡'

In [5]:
import string

class Tokenizer(): 
    
  def __init__(self):
    self.all_characters = all_characters
    self.n_characters = len(self.all_characters)
    
  def text_to_seq(self, string):
    seq = []
    for c in range(len(string)):
        try:
            seq.append(self.all_characters.index(string[c]))
        except:
            continue
    return seq

  def seq_to_text(self, seq):
    text = ''
    for c in range(len(seq)):
        text += self.all_characters[seq[c]]
    return text

tokenizer = Tokenizer()
tokenizer.n_characters

114

El tokenizer puede convertir una secuencia de texto en números, y al revés.

In [6]:
tokenizer.text_to_seq('señor, ¿qué tal?')

[28, 14, 100, 24, 27, 73, 94, 112, 26, 30, 104, 94, 29, 10, 21, 82]

In [7]:
tokenizer.seq_to_text([28, 14, 100, 24, 27, 73, 94, 112, 26, 30, 104, 94, 29, 10, 21, 82])

'señor, ¿qué tal?'

Ahora podemos tokenizar todo el texto.

In [8]:
text_encoded = tokenizer.text_to_seq(text)
print(text_encoded[700:750])
print(tokenizer.seq_to_text(text_encoded[700:750]))

[18, 94, 13, 14, 94, 21, 24, 94, 22, 10, 28, 94, 15, 18, 23, 24, 75, 94, 55, 14, 23, 18, 10, 94, 14, 23, 94, 28, 30, 94, 12, 10, 28, 10, 94, 30, 23, 10, 94, 10, 22, 10, 94, 26, 30, 14, 94, 25, 10, 28]
i de lo mas fino. Tenia en su casa una ama que pas


> 💡 Pese a que podemos implementar nuestra lógica de tokenización para trabajar a nivel de carácteres, cuando trabajamos con palabras completas el proceso puede complicarse. Es por esto que existen muchas herramientas que ya implementan este tipo de procesado (y muchos otros) que podemos utilizar. Un ejemplo, especialmente integrado con `Pytorch`, es la librería [torchtext](https://pytorch.org/text/).

## El *Dataset*

En primer lugar, vamos a separar nuestro texto en un conjunto de entrenamiento y otro de test. Cómo ya hemos hablado en posts anteriores, usaremos los datos de entrenamiento para entrenar nuestra red neuronal y los datos de test para calcular las métricas finales.

In [9]:
train_size = len(text_encoded) * 80 // 100 
train = text_encoded[:train_size]
test = text_encoded[train_size:]
print(len(text_encoded))
len(train), len(test)

1017582


(814065, 203517)

Para entrenar nuestra red, vamos a necesitar secuencias de texto de una longitud determinada. Podemos generar estas ventanas con la siguiente función

In [10]:
import random

def windows(text, window_size = 100):
    start_index = 0
    end_index = len(text) - window_size
    text_windows = []
    while start_index < end_index:
      text_windows.append(text[start_index:start_index+window_size+1])
      start_index += 1
    return text_windows

In [11]:
text_encoded_windows = windows(text_encoded, window_size=100)

print(len(text_encoded_windows))
print(text_encoded_windows[1016579])
print(text_encoded_windows[1016580])
print(text_encoded_windows[1016581])

1017482
[10, 23, 28, 24, 73, 96, 34, 94, 10, 21, 94, 15, 18, 23, 94, 25, 10, 27, 10, 18, 28, 94, 14, 23, 94, 28, 24, 22, 11, 27, 10, 73, 94, 14, 23, 94, 17, 30, 22, 24, 73, 94, 14, 23, 94, 28, 30, 14, 23, 24, 96, 39, 40, 47, 94, 38, 36, 38, 43, 44, 39, 44, 36, 37, 47, 50, 73, 94, 36, 38, 36, 39, 40, 48, 44, 38, 50, 94, 39, 40, 94, 47, 36, 94, 36, 53, 42, 36, 48, 36, 54, 44, 47, 47, 36, 73, 94, 40, 49, 94, 47]
[23, 28, 24, 73, 96, 34, 94, 10, 21, 94, 15, 18, 23, 94, 25, 10, 27, 10, 18, 28, 94, 14, 23, 94, 28, 24, 22, 11, 27, 10, 73, 94, 14, 23, 94, 17, 30, 22, 24, 73, 94, 14, 23, 94, 28, 30, 14, 23, 24, 96, 39, 40, 47, 94, 38, 36, 38, 43, 44, 39, 44, 36, 37, 47, 50, 73, 94, 36, 38, 36, 39, 40, 48, 44, 38, 50, 94, 39, 40, 94, 47, 36, 94, 36, 53, 42, 36, 48, 36, 54, 44, 47, 47, 36, 73, 94, 40, 49, 94, 47, 36]
[28, 24, 73, 96, 34, 94, 10, 21, 94, 15, 18, 23, 94, 25, 10, 27, 10, 18, 28, 94, 14, 23, 94, 28, 24, 22, 11, 27, 10, 73, 94, 14, 23, 94, 17, 30, 22, 24, 73, 94, 14, 23, 94, 28, 30, 1

Como puedes ver, hemos generado un número determinado de frases con la longitud especificada las cuales empiezan cada vez un carácter más a la derecha.

In [12]:
# print(tokenizer.seq_to_text((text_encoded_windows[0])))
# print()
# print(tokenizer.seq_to_text((text_encoded_windows[1])))
# print()
# print(tokenizer.seq_to_text((text_encoded_windows[2])))


# torch.tensor(
print(text_encoded_windows[10][:])
print("-"*10)
print(text_encoded_windows[10][:-1])
# torch.tensor(
print("-"*10)
print(text_encoded_windows[10][-1])

[40, 94, 39, 40, 94, 47, 36, 94, 48, 36, 49, 38, 43, 36, 96, 48, 18, 16, 30, 14, 21, 94, 13, 14, 94, 38, 14, 27, 31, 10, 23, 29, 14, 28, 94, 54, 10, 10, 31, 14, 13, 27, 10, 96, 96, 51, 53, 44, 48, 40, 53, 36, 94, 51, 36, 53, 55, 40, 96, 38, 36, 51, 44, 55, 56, 47, 50, 94, 1, 77, 94, 52, 30, 14, 94, 29, 27, 10, 29, 10, 94, 13, 14, 94, 21, 10, 94, 12, 24, 23, 13, 18, 12, 18, 24, 23, 94, 34, 94, 14, 19]
----------
[40, 94, 39, 40, 94, 47, 36, 94, 48, 36, 49, 38, 43, 36, 96, 48, 18, 16, 30, 14, 21, 94, 13, 14, 94, 38, 14, 27, 31, 10, 23, 29, 14, 28, 94, 54, 10, 10, 31, 14, 13, 27, 10, 96, 96, 51, 53, 44, 48, 40, 53, 36, 94, 51, 36, 53, 55, 40, 96, 38, 36, 51, 44, 55, 56, 47, 50, 94, 1, 77, 94, 52, 30, 14, 94, 29, 27, 10, 29, 10, 94, 13, 14, 94, 21, 10, 94, 12, 24, 23, 13, 18, 12, 18, 24, 23, 94, 34, 94, 14]
----------
19


Nuestro *dataset* de `Pytorch` se encargará de darnos cada una de estas frases, utilizando todos los carácteres excepto el último como entradas para la red y el último carácter como la etiqueta que usaremos durante el entrenamiento (la red deberá predecir la siguiente letra).

In [13]:
import torch

class CharRNNDataset(torch.utils.data.Dataset):
  def __init__(self, text_encoded_windows, train=True):
    self.text = text_encoded_windows
    self.train = train

  def __len__(self):
    return len(self.text)

  def __getitem__(self, ix):
    if self.train:
      return torch.tensor(self.text[ix][:-1]), torch.tensor(self.text[ix][-1])
    return torch.tensor(self.text[ix])

In [14]:
train_text_encoded_windows = windows(train)
test_text_encoded_windows = windows(test)

dataset = {
    'train': CharRNNDataset(train_text_encoded_windows),
    'val': CharRNNDataset(test_text_encoded_windows)
}

dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=512, shuffle=True, pin_memory=True),
    'val': torch.utils.data.DataLoader(dataset['val'], batch_size=2048, shuffle=False, pin_memory=True),
}

len(dataset['train']), len(dataset['val'])

(813965, 203417)

In [15]:
input, output = dataset['train'][10000]
print(tokenizer.seq_to_text(input))
print(output)

buen parecer, de quien el un tiempo anduvo enamorado, aunque segun se entiende, ella jamas lo supo n
tensor(18)


In [16]:
tokenizer.seq_to_text([output])

'i'

## Embeddings

Si bien hemos conseguido convertir nuestro texto a números, una red neuronal seguirá sin ser capaz de trabajar con nuestros datos ya que, como hemos visto en posts anteriores, éstos tienen que estar normalizados. Además, en función del `tokenizador` que utilicemos es posible que el  mismo carácter tenga asociados diferentes valores. Es por esto que necesitamos codificar nuestro texto de alguna manera. 

Una opción puede ser el `one-hot encoding`, al fin y al cabo podemos considerar cada letra como una categoría y que nuestra red nos de a la salida una distribución de probabilidad sobre todos los posibles carácteres. A continuación tienes un ejemplo de este tipo de codificación (utilizando palabras en vez de letras).

![](https://i0.wp.com/shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2018/01/one-hot-word-embedding-vectors.png?ssl=1)

A nuestra red le daremos a la entrada un vector que representará cada elemento en el vocabulario. Este vector tendrá una longitud igual al número de elementos diferentes en el vocabulario, y estará lleno de ceros excepto por una posición (la posición que ocupe el elemento en concreto dentro del vocabulario, la lista de elementos únicos). En nuestro caso podríamos optar por esta alternativa, ya que apenas tenemos un centenar de carácteres diferentes. Sin embargo, cuando trabajemos con palabras, nuestros vocabularios serán enormes (¿cuántas palabras hay en el diccionario?). Esto implica que trabajar con una codificación `one-hot` será muy costoso (vectores muy grandes) e ineficiente (prácticamente llenos de ceros). Es por esto que utilizamos una mejor codificación: los `embeddings`

![](https://i.stack.imgur.com/5gAnY.png)

Un embedding es una matriz con un número de filas igual al tamaño del vocabulario y un número de columnas que nosotros decidiremos. Cada fila en la matriz representará la codificación de una palabara (o carácter en nuestro ejemplo). A diferencia de la codificación `one-hot`, estos vectores son densos (pueden tener valores diferentes de cero en cualquier posición). Además, estos valores son aprendidos por la red neuronal, de manera que podrá representar los datos de la mejor forma posible para llevar a cabo la tarea. En la figura anterior tienes un ejemplo de un embedding entrenado, ¿observas algún patrón?. Efectivamente, palabras similares tienen representaciones similares. Además, cada columna del embedding tiene un significado que permite establecer relaciones entre las diferentes representaciones.

In [17]:
import numpy as np

man = np.array([0, 0, 0])
woman = np.array([1, 0, 0])
boy = np.array([0, 1, 0])
girl = np.array([1, 1, 0])
prince = np.array([0, 1, 1])
princess = np.array([1, 1, 1])
queen = np.array([1, 0, 1])
king = np.array([0, 0, 1])
monarch = np.array([0.5, 0.5, 1])

print((man - boy) + girl)

[1 0 0]


> ⚡ ¿Qué resultado obtienes al restar el vector `boy` al vector `man` y sumarle el vector `girl`?

En `Pytorch` tenemos esta capa implementada en la clase `torch.nn.Embedding`, y más adelante veremos como podemos utilizar `transfer learning` con embeddings pre-entrenados (lo cual nos dará una mejor representación de nuestro vocabulario desde el principio sin tener que entrenar esta capa).

In [18]:
class CharRNN(torch.nn.Module):
  def __init__(self, input_size, embedding_size=128, hidden_size=256, num_layers=2, dropout=0.2):
    super().__init__()
    self.encoder = torch.nn.Embedding(input_size, embedding_size)
    self.rnn = torch.nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, input_size)

  def forward(self, x):
    x = self.encoder(x)
    x, h = self.rnn(x)         
    y = self.fc(x[:,-1,:])
    return y

Nuestro modelo recibirá *batches* de frases con el índice de cada palabra que nos proporciona el `tokenizador`. A la salida tendremos una distribución de probabilidad sobre todos los posibles carácteres para cada frase del *batch*. Aquellos con mayor probabilidad serán los que la red cree que son buenos candidatos para seguir la frase recibida a la entrada.

In [19]:
print(tokenizer.n_characters)
a = torch.randint(0, tokenizer.n_characters, (64, 50))
print(a)
print(a.shape)

114
tensor([[110,  86,  72,  ..., 112, 104,  99],
        [ 98,  78,   5,  ...,  85,  56,  43],
        [ 42,  71, 107,  ...,  90,  89,  97],
        ...,
        [ 54, 102,  99,  ...,  13,  11,  93],
        [ 79,  14,  66,  ...,  50,  92,  41],
        [ 67,  56,  38,  ...,  56,  10, 101]])
torch.Size([64, 50])


In [20]:
model = CharRNN(input_size=tokenizer.n_characters, embedding_size = 128)
outputs = model(torch.randint(0, tokenizer.n_characters, (64, 50)))
# outputs = model(a)
outputs.shape

torch.Size([64, 114])

## Entrenamiento

In [21]:
from tqdm import tqdm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f}")
        bar = tqdm(dataloader['val'])
        val_loss = []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f}")

def predict(model, X):
    model.eval() 
    with torch.no_grad():
        X = torch.tensor(X).to(device)
        pred = model(X.unsqueeze(0))
        return pred

cuda


In [22]:
# x = torch.tensor([1, 2, 3, 4])
# print(x.shape)
# print(torch.unsqueeze(x, 0))
# print(torch.unsqueeze(x, 0).shape)
# print(torch.unsqueeze(x, 1))
# print(torch.unsqueeze(x, 1).shape)

In [23]:
model = CharRNN(input_size=tokenizer.n_characters)
fit(model, dataloader, epochs=20)

loss 1.57818:   0%|          | 1/1590 [00:00<03:34,  7.42it/s]

Epoch 1/20 loss 1.85746 val_loss 1.58761


loss 1.36547:   0%|          | 1/1590 [00:00<03:47,  6.99it/s]

Epoch 2/20 loss 1.50182 val_loss 1.44527


loss 1.33189:   0%|          | 1/1590 [00:00<03:24,  7.75it/s]

...

Epoch 18/20 loss 1.13372 val_loss 1.24238


loss 1.12582:   0%|          | 1/1590 [00:00<03:25,  7.73it/s]

Epoch 19/20 loss 1.12680 val_loss 1.24663


val_loss 1.24455: 100%|██████████| 100/100 [00:09<00:00, 11.06it/s]

Epoch 20/20 loss 1.12164 val_loss 1.24455


## Generando texto

Una vez hemos entrenado nuestro modelo, podemos darle una frase para que genere la siguiente letra.

In [24]:
X_new = "En un lugar de la mancha, "
#X_new = "Cuando en lo profundo de mi conciencia, "
X_new_encoded = tokenizer.text_to_seq(X_new)
print(X_new_encoded)
y_pred = predict(model, X_new_encoded)
print(y_pred.shape)
y_pred = torch.argmax(y_pred, axis=1)[0].item()
tokenizer.seq_to_text([y_pred])

[40, 23, 94, 30, 23, 94, 21, 30, 16, 10, 27, 94, 13, 14, 94, 21, 10, 94, 22, 10, 23, 12, 17, 10, 73, 94]
torch.Size([1, 114])


'y'

Podemos generar más letras añadiendo las predicciones como parte de la entrada, generando texto letra a letra.

In [25]:
for i in range(100):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = torch.argmax(y_pred, axis=1)[0].item()
  X_new += tokenizer.seq_to_text([y_pred])

X_new

'En un lugar de la mancha, y que es la mano a la mano a la mano a la mano a la mano a la mano a la mano a la mano a la mano a l'

Cómo puedes ver el text generado puede ser repetitivo si simplemente nos quedamos con la letra con mayor probabilidad. Para generar texto con mayor variedad, es común elegir de manera aleatoria una letra de entre las que tienen mayor probabilidad.

In [26]:
temp=1
for i in range(1000):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = y_pred.view(-1).div(temp).exp()
  top_i = torch.multinomial(y_pred, 1)[0]
  predicted_char = tokenizer.all_characters[top_i]
  X_new += predicted_char

print(X_new)

En un lugar de la mancha, y que es la mano a la mano a la mano a la mano a la mano a la mano a la mano a la mano a la mano a la Zirera; y como le quitaje que habian recibido, se viendo en nuestros primeros, y el tan con ejercicio de poco manera, al cual sabia entro en todo; que es procura que no estaba bien, y luego no hay culpado del nombre vuestra gracia. Pues ¿que piensa, dijo:
-Ahora oi querria que asi le hubiese, pues fue en mi mentiro un general de los bajes deste gahavilitado por el, hare tiempo que la tiene otra cosa que de hoy de simplado en todo el cuerto o amigo de bonco a buscarle de los hombres sin duda que en Espana. En
remotimiendo hasta la son animas, hacia mirando, tantas razones que fue que un sueltamente gustaron de las piedras. Creciome Sancho Panza, y ella ya el rocar en la sepultura y parecer a su pena, con la miranda y el fingido ni hecho se leyendo un hombrado, para conocimiento; mas decia a los pies padres, ni decir las aventuras que al tiempos y discursos pasi

## Resumen

En este post hemos aprendido cómo implementar y entrenar una `red neuronal recurrente` para generar texto como si fuese Miguel de Cervantes. Para ello hemos utilizado su libro *Don Quijote de la Mancha* como dataset. En primer lugar, transformamos el texto en números gracias al proceso de la `tokenización`. Después, codificamos cada carácter en el dataset utilizando una capa `embedding`, que permitirá a la red neuronal encontrar la mejor representación posible de los datos para llevar a cabo su tarea. Para generar texto, le pedimos a la red que nos de una distribución de probabilidad sobre todos los posible carácteres a partir de una frase que le damos a la entrada. Utilizaremos esta distribución para seleccionar un carácter que siga con la frase de manera convincente. Podemos repetir este proceso para generar secuencias más largas.